In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import datetime

# Define the Data Sources

## Demand

In [ ]:

baltic_energy_load = pd.read_csv('/home/ctang/ttmp/Datathon/data_and_starter_code/data/baltic_energy_load.csv')

In [ ]:
baltic_energy_load['Date'] = baltic_energy_load['MTU (CET/CEST)'].astype(str)
baltic_energy_load['Date'] = baltic_energy_load['Date'].str[:10]
baltic_energy_load['Date'] = pd.to_datetime(baltic_energy_load['Date'], format="%d/%m/%Y")
baltic_energy_load = baltic_energy_load.drop(columns=['MTU (CET/CEST)'])
baltic_energy_load = baltic_energy_load.drop(columns=['Day-ahead Total Load Forecast (MW)'])

In [ ]:
# rename Area to country
baltic_energy_load = baltic_energy_load.rename(columns={'Area': 'country'})

# rename Actual Total Load (MW) to demand
baltic_energy_load = baltic_energy_load.rename(columns={'Actual Total Load (MW)': 'demand'})

# set index to Date
baltic_energy_load['demand'] = baltic_energy_load['demand'].astype(float)

In [ ]:
start_date = '2024-09-01'
end_date = '2024-11-30'

# sum spring months
demand_df = baltic_energy_load[(baltic_energy_load['Date'] >= start_date) & (baltic_energy_load['Date'] <= end_date)]
demand_df = demand_df.drop(columns=['Date'])
demand_df = demand_df.groupby(['country']).sum().reset_index()
demand_df

## Capacity

In [ ]:
winter_capacity_dict = {}
spring_capacity_dict = {}
summer_capacity_dict = {}
fall_capacity_dict = {}

winter_capacity_dict['Estonia'] = {
    'Solar': 150000,
    'Wind': 300000,
    'Biomass': 150000,
    'Hydro': 1000,
    'Gas': 500000,
    'Oil Shale': 11.7e6 / 4,
    'Peat': 50000,
    'Coal': 100000
}
spring_capacity_dict['Estonia'] = {
    'Solar': 500000,
    'Wind': 200000,
    'Biomass': 100000,
    'Hydro': 1000,
    'Gas': 100000,
    'Oil Shale': 11.7e6 / 4,
    'Peat': 40000,
    'Coal': 150000
}
summer_capacity_dict['Estonia'] = {
    'Solar': 600000,
    'Wind': 300000,
    'Biomass': 80000,
    'Hydro': 1000,
    'Gas': 300000,
    'Oil Shale': 11.7e6 / 4,
    'Peat': 30000,
    'Coal': 60000
}
fall_capacity_dict['Estonia'] = {
    'Solar': 350000,
    'Wind': 350000,
    'Biomass': 120000,
    'Hydro': 1000,
    'Gas': 400000,
    'Oil Shale': 11.7e6 / 4,
    'Peat': 40000,
    'Coal': 80000
}
winter_capacity_dict['Latvia'] = {
    'Solar': 100000,
    'Wind': 50000,
    'Biomass': 150000,
    'Hydro': 250000,
    'Gas': 250000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 80000
}
spring_capacity_dict['Latvia'] = {
    'Solar': 200000,
    'Wind': 60000,
    'Biomass': 100000,
    'Hydro': 450000,
    'Gas': 250000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 70000
}
summer_capacity_dict['Latvia'] = {
    'Solar': 250000,
    'Wind': 40000,
    'Biomass': 90000,
    'Hydro': 450000,
    'Gas': 200000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 60000
}
fall_capacity_dict['Latvia'] = {
    'Solar': 200000,
    'Wind': 50000,
    'Biomass': 100000,
    'Hydro': 350000,
    'Gas': 200000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 70000
}
winter_capacity_dict['Lithuania'] = {
    'Solar': 400000,
    'Wind': 300000,
    'Biomass': 120000,
    'Hydro': 100000,
    'Gas': 300000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 90000
}
spring_capacity_dict['Lithuania'] = {
    'Solar': 200000,
    'Wind': 400000,
    'Biomass': 90000,
    'Hydro': 200000,
    'Gas': 250000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 80000
}
summer_capacity_dict['Lithuania'] = {
    'Solar': 600000,
    'Wind': 300000,
    'Biomass': 70000,
    'Hydro': 150000,
    'Gas': 200000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 70000
}
fall_capacity_dict['Lithuania'] = {
    'Solar': 500000,
    'Wind': 300000,
    'Biomass': 100000,
    'Hydro': 50000,
    'Gas': 250000,
    'Oil Shale': 0,
    'Peat': 0,
    'Coal': 80000
}

In [ ]:
# convert dict to dataframe
winter_capacity_df = pd.DataFrame.from_dict(winter_capacity_dict, orient='index')
spring_capacity_df = pd.DataFrame.from_dict(spring_capacity_dict, orient='index')
summer_capacity_df = pd.DataFrame.from_dict(summer_capacity_dict, orient='index')
fall_capacity_df = pd.DataFrame.from_dict(fall_capacity_dict, orient='index')

In [ ]:
# reformat columns to be ['source', 'country', 'capacity']
winter_capacity_df = winter_capacity_df.stack().reset_index()
winter_capacity_df.columns = ['country', 'source', 'capacity']
spring_capacity_df = spring_capacity_df.stack().reset_index()
spring_capacity_df.columns = ['country', 'source', 'capacity']
summer_capacity_df = summer_capacity_df.stack().reset_index()
summer_capacity_df.columns = ['country', 'source', 'capacity']
fall_capacity_df = fall_capacity_df.stack().reset_index()
fall_capacity_df.columns = ['country', 'source', 'capacity']

## Energy Cost

In [ ]:
gen_cost_dict = {}
gen_cost_dict['Estonia'] = {
    'Solar': 90,
    'Wind': 70,
    'Biomass': 90,
    'Hydro': 70,
    'Gas': 80,
    'Oil Shale': 60,
    'Coal': 100
}
gen_cost_dict['Latvia'] = {
    'Solar': 90,
    'Wind': 70,
    'Biomass': 80,
    'Hydro': 50,
    'Gas': 75,
    'Oil Shale': 1,
    'Peat': 85,
    'Coal': 90
}
gen_cost_dict['Lithuania'] = {
    'Solar': 80,
    'Wind': 70,
    'Biomass': 80,
    'Hydro': 50,
    'Gas': 70,
    'Oil Shale': 1,
    'Peat': 85,
    'Coal': 90
}

#convert to dataframe
gen_cost = pd.DataFrame.from_dict(gen_cost_dict, orient='index')

In [ ]:
# reformat dataaframe to have columns ['country', 'source', 'cost']
gen_cost = gen_cost.reset_index()
gen_cost = gen_cost.melt(id_vars=['index'], value_vars=['Solar', 'Wind', 'Biomass', 'Hydro', 'Gas', 'Oil Shale', 'Peat', 'Coal'])
gen_cost = gen_cost.rename(columns={'index': 'country', 'variable': 'source', 'value': 'cost'})
cost_df = gen_cost

## Trasmission Cost

In [ ]:
trans_cost = {}
trans_cost[('Estonia', 'Lithuania')] = 13.29
trans_cost[('Estonia', 'Latvia')] = 13.29
trans_cost[('Latvia', 'Lithuania')] = 13.29
trans_cost[('Lithuania', 'Estonia')] = 13.29
trans_cost[('Latvia', 'Estonia')] = 13.29
trans_cost[('Lithuania', 'Latvia')] = 13.29

# convert to a dataframe
trans_cost_df = pd.DataFrame(trans_cost.items(), columns=['country_pair', 'cost'])

# convert country_pair to two columns
trans_cost_df[['from', 'to']] = pd.DataFrame(trans_cost_df['country_pair'].tolist(), index=trans_cost_df.index)

# optionally drop the original 'country_pair' column
trans_cost_df = trans_cost_df.drop(columns='country_pair')

print(trans_cost_df)

## Transmission Cap

In [ ]:
transmission_data = pd.read_csv("/home/ctang/ttmp/Datathon/data_and_starter_code/data/transmission.csv")

In [ ]:
# choose data with Out Area of BZN|EE or BZN|LV or BZN|LT
transmission_data = transmission_data[(transmission_data['Out Area'] == 'BZN|EE') | (transmission_data['Out Area'] == 'BZN|LV') | (transmission_data['Out Area'] == 'BZN|LT')]
transmission_data = transmission_data[(transmission_data['In Area'] == 'BZN|EE') | (transmission_data['In Area'] == 'BZN|LV') | (transmission_data['In Area'] == 'BZN|LT')]

In [ ]:
# map out area BZN|LV to Lithuania, BZN|LT to Latvia
transmission_data['Out Area'] = transmission_data['Out Area'].map({'BZN|LT': 'Lithuania', 'BZN|LV': 'Latvia', 'BZN|EE': 'Estonia'})
transmission_data = transmission_data.rename(columns={'Out Area': 'from'})

# map in area BZN|EE to Estonia
transmission_data['In Area'] = transmission_data['In Area'].map({'BZN|EE': 'Estonia', 'BZN|LV': 'Latvia', 'BZN|LT': 'Lithuania'})
transmission_data = transmission_data.rename(columns={'In Area': 'to'})

In [ ]:
transmission_data

In [ ]:
# find max Physical Flow from Lithuania to Estonia and from Latvia to Estonia
transmission_data['Physical Flow (MW)'] = transmission_data['Physical Flow (MW)'].astype(float)
transmission_data['Physical Flow (MW)'] = transmission_data['Physical Flow (MW)'].abs()
transmission_data = transmission_data.drop(columns=['MTU'])

In [ ]:
trans_cap_df = transmission_data.groupby(['from', 'to']).max().reset_index()
trans_cap_df = trans_cap_df.rename(columns={'Physical Flow (MW)': 'capacity'})
trans_cap_df

In [ ]:
import itertools

# Get all unique countries
countries = pd.unique(trans_cap_df[['from', 'to']].values.ravel())

# Create all possible (from, to) pairs, excluding self-to-self
all_pairs = pd.DataFrame(
    [(f, t) for f, t in itertools.product(countries, countries) if f != t],
    columns=['from', 'to']
)

# Merge with the original DataFrame and fill missing capacities with 0
trans_cap_df = all_pairs.merge(trans_cap_df, on=['from', 'to'], how='left').fillna({'capacity': 0})

# multiply capacity by 90
trans_cap_df['capacity'] = trans_cap_df['capacity'] * 90

## Emissions

In [ ]:
energy_type = pd.read_csv("/home/ctang/ttmp/Datathon/data_and_starter_code/data/baltic_energy_type.csv")
carbon_intensity = pd.read_csv("/home/ctang/ttmp/Datathon/data_and_starter_code/data/carbon_intensity_countries.csv")

In [ ]:
carbon_intensity = carbon_intensity.drop(columns=['Zone id', 'Carbon intensity gCO‚ÇÇeq/kWh (direct)', 'Data source', 'Carbon-free energy percentage (CFE%)', 'Renewable energy percentage (RE%)'])

In [ ]:
# convert to date
energy_type["MTU (CET/CEST)"] = pd.to_datetime(energy_type["MTU (CET/CEST)"], format="%d/%m/%Y")
energy_type = energy_type.rename(columns={'MTU (CET/CEST)': 'Date'})

carbon_intensity["Datetime (UTC)"] = pd.to_datetime(carbon_intensity["Datetime (UTC)"], format="%Y-%m-%d")
carbon_intensity = carbon_intensity.rename(columns={'Datetime (UTC)': 'Date'})

In [ ]:
# rename Area to Country in energy_type
energy_type = energy_type.rename(columns={'Area': 'Country'})

# join energy_type and carbon_intensity on Date and Country
df = energy_type.merge(carbon_intensity, on=['Date', 'Country'])
df = df.rename(columns={'Carbon intensity gCO‚ÇÇeq/kWh (Life cycle)': 'carbon_intensity gCO2eq/MWh'})
df['carbon_intensity gCO2eq/MWh'] = df['carbon_intensity gCO2eq/MWh'] * 1000

In [ ]:
df['Generation (MW)'] = pd.to_numeric(df['Generation (MW)'], errors='coerce')
df['carbon_intensity gCO2eq/MWh'] = pd.to_numeric(df['carbon_intensity gCO2eq/MWh'], errors='coerce')

df['Emissions (g CO2eq)'] = (df['Generation (MW)'] * df['carbon_intensity gCO2eq/MWh'])

In [ ]:
# create feature vectors
X = df.pivot_table(
    index=['Date', 'Country'],
    columns='Production Type',
    values='Generation (MW)',
    aggfunc='sum',
    fill_value=0
)
y = df.groupby(['Date', 'Country'])['Emissions (g CO2eq)'].sum()
X, y = X.align(y, join='inner', axis=0)

In [ ]:
# train linear model on X,y
from LinearModel import *
model = test_linear_model(X,y)

In [ ]:
emissions_dict = {
    'Solar': 3.721e4,
    'Wind': 6.563e4,
    'Biomass': 4.119e5,
    'Hydro': 3.35e4,
    'Gas': 4.55e5,
    'Oil Shale': 7.167e5,
    'Peat': 1.02e7,
    'Coal': 3.485e5,
}

# convert to dataframe
emissions_df = pd.DataFrame.from_dict(emissions_dict, orient='index')
emissions_df = emissions_df.reset_index()
emissions_df.columns = ['source', 'emission']
emissions_df

# Define the LP

In [ ]:
capacity_df = fall_capacity_df

In [ ]:
from pyomo.environ import *

# Initialize sets
countries = demand_df['country'].unique().tolist()
sources = cost_df['source'].unique().tolist()
country_pairs = list(trans_cost_df[['from', 'to']].itertuples(index=False, name=None))

# Model
model = ConcreteModel()
model.COUNTRIES = Set(initialize=countries)
model.SOURCES = Set(initialize=sources)
model.PAIRS = Set(initialize=country_pairs, dimen=2)

# Parameters
C = cost_df.set_index(['source', 'country'])['cost'].to_dict()
K = capacity_df.set_index(['source', 'country'])['capacity'].to_dict()
Q = trans_cost_df.set_index(['from', 'to'])['cost'].to_dict()
L = trans_cap_df.set_index(['from', 'to'])['capacity'].to_dict()
D = demand_df.set_index('country')['demand'].to_dict()
E = emissions_df.set_index('source')['emission'].to_dict()

renewable_sources = ['Solar', 'Wind', 'Biomass', 'Hydro']
alpha = 1
beta = 2.5e-5 # low carbon tax

# Scalars
α = alpha
β = beta
w = 0.7
r_target = {
    'Estonia': 0.42,
    'Latvia': 0.42,
    'Lithuania': 0.42
}
renewable_set = set(renewable_sources)

# Variables
model.P = Var(model.SOURCES, model.COUNTRIES, domain=NonNegativeReals)   # P_{s,c}
model.T = Var(model.PAIRS, domain=NonNegativeReals)                      # T_{c1 -> c2}

# Objective
def obj_rule(model):
    cost_term = sum(α * C[s,c] * model.P[s,c] for s in sources for c in countries if (s,c) in C)
    trans_term = sum(α * Q[c1,c2] * model.T[c1,c2] for (c1,c2) in country_pairs)
    emissions_term = sum(β * E[s] * model.P[s,c] for s in sources for c in countries if (s,c) in E)
    return cost_term + trans_term + emissions_term
model.OBJ = Objective(rule=obj_rule, sense=minimize)

# Demand constraint
def demand_constraint(model, c):
    gen = sum(model.P[s, c] for s in sources if (s, c) in K)
    inflow = sum(model.T[i, c] for (i, j) in country_pairs if j == c)
    outflow = sum(model.T[c, j] for (i, j) in country_pairs if i == c)
    return gen + inflow - outflow >= D[c]
model.demand = Constraint(model.COUNTRIES, rule=demand_constraint)

# Capacity constraint
model.cap_constraint = ConstraintList()
for (s, c) in K:
    model.cap_constraint.add(model.P[s, c] <= K[s, c] * 2)

# Transmission constraint
model.trans_constraint = ConstraintList()
for (c1, c2) in country_pairs:
    model.trans_constraint.add(model.T[c1, c2] <= L[c1, c2] * 2)

# Renewable constraint
def renewable_constraint(model, c):
    total_renewables = sum(model.P[s,c] for s in renewable_set if (s,c) in K)
    total_production = sum(model.P[s,c] for s in sources if (s,c) in K)
    return total_renewables >= r_target[c] * total_production
model.renewable_constraint = ConstraintList()
for c in countries:
    model.renewable_constraint.add(renewable_constraint(model, c))

opt = SolverFactory('cbc')
results = opt.solve(model)
# check if solution exists
if results.solver.termination_condition == TerminationCondition.optimal:
    print("Solution found")
    generation = {(s,c): model.P[s,c].value for s in sources for c in countries if (s,c) in K}
    transmission = {(c1,c2): model.T[c1,c2].value for (c1,c2) in country_pairs}

    for (c1,c2) in country_pairs:
        print(f"From {c1} to {c2}: {transmission[c1,c2]} MW")
    for s in sources:
        for c in countries:
            print(f"Generation of {s} in {c}: {generation[s,c]} MW")
else:
    assert False, "No solution found"

In [ ]:
results.write()

In [ ]:
# Define consistent colors for each source
color_palette = plt.get_cmap('tab20')  # or try 'Set3', 'Paired', 'Dark2', etc.
source_list = sorted(set(s for s, _ in K))
color_map = {source: color_palette(i % 20) for i, source in enumerate(source_list)}

# Create side-by-side pie charts for each country
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

for i, country in enumerate(countries):
    # Calculate total generation for the country
    total_gen = sum(generation[source, country] for source in sources if (source, country) in K)

    # Filter out sources contributing less than 1% of total generation
    gen = {
        source: generation[source, country]
        for source in sources
        if (source, country) in K and generation[source, country] / total_gen > 0.005
    }

    # Plot pie chart with consistent colors
    axs[i].pie(
        gen.values(),
        labels=gen.keys(),
        autopct='%1.1f%%',
        startangle=90,
        counterclock=False,
        wedgeprops={'edgecolor': 'white'},
        colors=[color_map[s] for s in gen.keys()]
    )
    axs[i].set_title(country, fontsize=12)

# Add an overall title and adjust layout
fig.suptitle('2024 Electricity Generation by Source', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
# calculate total emissions
total_emissions = 0
for s in sources:
    for c in countries:
        total_emissions += generation[s,c] * emissions_df.loc[emissions_df['source'] == s, 'emission'].values[0]

# prin tin scientific notation
print(f'Total emissions: {total_emissions:.2e} g CO2eq')

In [ ]:
print("Total demand:", sum(D.values()))
print("Total generation capacity:", sum(K.values()))
for c in countries:
    print(f"Required renewable generation in {c}: {r_target[c] * D[c]}")

In [ ]:
# calculate total generation by country
total_generation = {c: sum(generation[s,c] for s in sources if (s,c) in K) for c in countries}
print("Total generation by country:", total_generation)

In [ ]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def run_lp_model(α, β, C, K, Q, L, D, E, renewable_sources, r_target, scaling_factors=None):
    """
    Run the LP model with given parameters
    
    scaling_factors: Optional dictionary with scaling factors for capacity constraints
    """
    # Initialize sets
    countries = list(D.keys())
    sources = list(set(s for s, _ in C.keys()))
    country_pairs = list(Q.keys())
    
    # Apply scaling factors to capacities if provided
    if scaling_factors:
        if 'capacity' in scaling_factors:
            K = {k: v * scaling_factors['capacity'] for k, v in K.items()}
        if 'transmission' in scaling_factors:
            L = {k: v * scaling_factors['transmission'] for k, v in L.items()}
        if 'demand' in scaling_factors:
            D = {k: v * scaling_factors['demand'] for k, v in D.items()}
    
    # Model
    model = ConcreteModel()
    model.COUNTRIES = Set(initialize=countries)
    model.SOURCES = Set(initialize=sources)
    model.PAIRS = Set(initialize=country_pairs, dimen=2)
    
    renewable_set = set(renewable_sources)
    
    # Variables
    model.P = Var(model.SOURCES, model.COUNTRIES, domain=NonNegativeReals)
    model.T = Var(model.PAIRS, domain=NonNegativeReals)
    
    # Objective
    def obj_rule(model):
        cost_term = sum(α * C[s,c] * model.P[s,c] for s in sources for c in countries if (s,c) in C)
        trans_term = sum(α * Q[c1,c2] * model.T[c1,c2] for (c1,c2) in country_pairs)
        emissions_term = sum(β * E[s] * model.P[s,c] for s in sources for c in countries if (s,c) in E)
        return cost_term + trans_term + emissions_term
    model.OBJ = Objective(rule=obj_rule, sense=minimize)
    
    # Demand constraint
    def demand_constraint(model, c):
        gen = sum(model.P[s, c] for s in sources if (s, c) in K)
        inflow = sum(model.T[i, c] for (i, j) in country_pairs if j == c)
        outflow = sum(model.T[c, j] for (i, j) in country_pairs if i == c)
        return gen + inflow - outflow >= D[c]
    model.demand = Constraint(model.COUNTRIES, rule=demand_constraint)
    
    # Capacity constraint
    model.cap_constraint = ConstraintList()
    for (s, c) in K:
        model.cap_constraint.add(model.P[s, c] <= K[s, c] * 2.5)
    
    # Transmission constraint
    model.trans_constraint = ConstraintList()
    for (c1, c2) in country_pairs:
        model.trans_constraint.add(model.T[c1, c2] <= L[c1, c2])
    
    # Renewable constraint
    def renewable_constraint(model, c):
        total_renewables = sum(model.P[s,c] for s in renewable_set if (s,c) in K)
        total_production = sum(model.P[s,c] for s in sources if (s,c) in K)
        return total_renewables >= r_target[c] * total_production
    model.renewable_constraint = ConstraintList()
    for c in countries:
        model.renewable_constraint.add(renewable_constraint(model, c))
    
    # Solve
    opt = SolverFactory('cbc')
    results = opt.solve(model)
    
    # Process results
    if results.solver.termination_condition == TerminationCondition.optimal:
        # total cost is sum of generation cost and transmission cost
        generation_cost = 0
        for s in sources:
            for c in countries:
                term = C[s,c] * model.P[s,c].value
                # test if term is not NaN
                if not np.isnan(term):
                    generation_cost += term
        transmission_cost = sum(Q[c1,c2] * model.T[c1,c2].value for (c1,c2) in country_pairs)
        print("Generation cost:", generation_cost)
        print("Transmission cost:", transmission_cost)
        total_cost = generation_cost + transmission_cost
        generation = {(s,c): model.P[s,c].value for s in sources for c in countries if (s,c) in K}
        transmission = {(c1,c2): model.T[c1,c2].value for (c1,c2) in country_pairs}
        
        # Calculate metrics
        production_by_country = {c: sum(generation.get((s,c), 0) for s in sources) for c in countries}
        renewable_by_country = {c: sum(generation.get((s,c), 0) for s in renewable_sources if (s,c) in K) for c in countries}
        renewable_percentage = {c: renewable_by_country[c]/production_by_country[c] if production_by_country[c] > 0 else 0 
                               for c in countries}
        
        emissions_by_country = {c: sum(E[s] * generation.get((s,c), 0) for s in sources if (s,c) in K and s in E) 
                               for c in countries}
        total_emissions = sum(emissions_by_country.values())
        
        net_import = {c: sum(transmission.get((c1,c), 0) for c1 in countries if (c1,c) in country_pairs) - 
                        sum(transmission.get((c,c2), 0) for c2 in countries if (c,c2) in country_pairs) 
                     for c in countries}
        
        return {
            'status': 'optimal',
            'total_cost': total_cost,
            'generation': generation,
            'transmission': transmission,
            'production_by_country': production_by_country,
            'renewable_percentage': renewable_percentage,
            'emissions_by_country': emissions_by_country,
            'total_emissions': total_emissions,
            'net_import': net_import
        }
    else:
        return {'status': 'infeasible'}

def sensitivity_analysis(C, K, Q, L, D, E, renewable_sources, r_target):
    """
    Perform comprehensive sensitivity analysis on the LP model
    """
    results = {}
    
    # 1. Cost vs Emissions Trade-off (α vs β)
    # alpha be logspace between 0, 1e-10, and 1e10
    alpha_values = [0.0, 1e-10, 1e-5, 1e-2, 1, 1e2, 1e5, 1e10]
    beta_values = [0.0, 1e-10, 1e-5, 1e-2, 1, 1e2, 1e5, 1e10]
    
    alpha_beta_results = []
    
    for α in alpha_values:
        for β in beta_values:
            print(f"Running model with α={α}, β={β}")
            result = run_lp_model(α, β, C, K, Q, L, D, E, renewable_sources, r_target)
            print(result)
            if result['status'] == 'optimal':
                alpha_beta_results.append({
                    'alpha': α,
                    'beta': β,
                    'total_cost': result['total_cost'],
                    'total_emissions': result['total_emissions'],
                    'renewable_percentage': {c: result['renewable_percentage'][c] for c in result['renewable_percentage']}
                })
    
    results['alpha_beta_sensitivity'] = pd.DataFrame(alpha_beta_results)
    
    # 2. Capacity Constraint Sensitivity
    capacity_scaling_factors = [0.8, 0.9, 1.0, 1.1, 1.2]
    capacity_results = []
    
    for scale in capacity_scaling_factors:
        print(f"Running model with capacity scaling={scale}")
        result = run_lp_model(1.0, 2.5e-5, C, K, Q, L, D, E, renewable_sources, r_target, 
                              scaling_factors={'capacity': scale})
        if result['status'] == 'optimal':
            capacity_results.append({
                'scaling_factor': scale,
                'total_cost': result['total_cost'],
                'total_emissions': result['total_emissions'],
                'production_by_country': {c: result['production_by_country'][c] for c in result['production_by_country']}
            })
    
    results['capacity_sensitivity'] = pd.DataFrame(capacity_results)
    
    # 3. Transmission Capacity Sensitivity
    transmission_scaling_factors = [0.7, 0.85, 1.0, 1.15, 1.3]
    transmission_results = []
    
    for scale in transmission_scaling_factors:
        print(f"Running model with transmission scaling={scale}")
        result = run_lp_model(1.0, 2.5e-5, C, K, Q, L, D, E, renewable_sources, r_target, 
                              scaling_factors={'transmission': scale})
        if result['status'] == 'optimal':
            transmission_results.append({
                'scaling_factor': scale,
                'total_cost': result['total_cost'],
                'total_emissions': result['total_emissions'],
                'net_import': {c: result['net_import'][c] for c in result['net_import']}
            })
    
    results['transmission_sensitivity'] = pd.DataFrame(transmission_results)
    
    # 4. Demand Sensitivity
    demand_scaling_factors = [0.85, 0.925, 1.0, 1.075, 1.15]
    demand_results = []
    
    for scale in demand_scaling_factors:
        print(f"Running model with demand scaling={scale}")
        result = run_lp_model(1.0, 2.5e-5, C, K, Q, L, D, E, renewable_sources, r_target, 
                              scaling_factors={'demand': scale})
        if result['status'] == 'optimal':
            demand_results.append({
                'scaling_factor': scale,
                'total_cost': result['total_cost'],
                'total_emissions': result['total_emissions'],
                'production_by_country': {c: result['production_by_country'][c] for c in result['production_by_country']}
            })
    
    results['demand_sensitivity'] = pd.DataFrame(demand_results)
    
    # 5. Renewable Target Sensitivity
    renewable_targets = [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0]
    renewable_results = []
    
    for target in renewable_targets:
        modified_targets = {c: target for c in r_target}
        print(f"Running model with renewable target={target}")
        result = run_lp_model(1.0, 2.5e-5, C, K, Q, L, D, E, renewable_sources, modified_targets)
        if result['status'] == 'optimal':
            renewable_results.append({
                'target': target,
                'total_cost': result['total_cost'],
                'total_emissions': result['total_emissions'],
                'renewable_percentage': {c: result['renewable_percentage'][c] for c in result['renewable_percentage']}
            })
    
    results['renewable_target_sensitivity'] = pd.DataFrame(renewable_results)
    
    return results

def visualize_sensitivity_analysis(results):
    """
    Create visualizations for sensitivity analysis results
    """
    # 1. Cost vs Emissions Trade-off
    if 'alpha_beta_sensitivity' in results:
        df = results['alpha_beta_sensitivity']
        
        # Create a pivot table for heatmap
        cost_pivot = df.pivot_table(index='alpha', columns='beta', values='total_cost')
        emissions_pivot = df.pivot_table(index='alpha', columns='beta', values='total_emissions')
        
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        
        sns.heatmap(cost_pivot, annot=True, fmt='.2e', cmap='viridis', ax=axes[0])
        axes[0].set_title('Total Cost Sensitivity to α and β')
        axes[0].set_xlabel('β (Carbon Tax)')
        axes[0].set_ylabel('α (Cost Coefficient)')
        
        sns.heatmap(emissions_pivot, annot=True, fmt='.2e', cmap='viridis', ax=axes[1])
        axes[1].set_title('Total Emissions Sensitivity to α and β')
        axes[1].set_xlabel('β (Carbon Tax)')
        axes[1].set_ylabel('α (Cost Coefficient)')
        
        plt.tight_layout()
        plt.savefig('alpha_beta_sensitivity.png')
        plt.close()
        
        # Pareto frontier
        plt.figure(figsize=(10, 6))
        plt.scatter(df['total_emissions'], df['total_cost'], c=df['beta'], cmap='viridis')
        plt.colorbar(label='β (Carbon Tax)')
        plt.title('Pareto Frontier: Cost vs. Emissions')
        plt.xlabel('Total Emissions')
        plt.ylabel('Total Cost')
        plt.grid(True, alpha=0.3)
        plt.savefig('pareto_frontier.png')
        plt.close()
    
    # 2. Capacity Constraint Sensitivity
    if 'capacity_sensitivity' in results:
        df = results['capacity_sensitivity']
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        axes[0].plot(df['scaling_factor'], df['total_cost'], marker='o', linewidth=2)
        axes[0].set_title('Impact of Capacity Scaling on Total Cost')
        axes[0].set_xlabel('Capacity Scaling Factor')
        axes[0].set_ylabel('Total Cost')
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(df['scaling_factor'], df['total_emissions'], marker='o', linewidth=2)
        axes[1].set_title('Impact of Capacity Scaling on Total Emissions')
        axes[1].set_xlabel('Capacity Scaling Factor')
        axes[1].set_ylabel('Total Emissions')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('capacity_sensitivity.png')
        plt.close()
        
        # Production by country
        countries = list(df['production_by_country'].iloc[0].keys())
        production_data = {c: [row['production_by_country'][c] for _, row in df.iterrows()] for c in countries}
        
        plt.figure(figsize=(12, 6))
        for c in countries:
            plt.plot(df['scaling_factor'], production_data[c], marker='o', label=c)
        plt.title('Production by Country vs. Capacity Scaling')
        plt.xlabel('Capacity Scaling Factor')
        plt.ylabel('Total Production (MWh)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig('production_by_country_capacity.png')
        plt.close()
    
    # 3. Transmission Capacity Sensitivity
    if 'transmission_sensitivity' in results:
        df = results['transmission_sensitivity']
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        axes[0].plot(df['scaling_factor'], df['total_cost'], marker='o', linewidth=2)
        axes[0].set_title('Impact of Transmission Capacity on Total Cost')
        axes[0].set_xlabel('Transmission Capacity Scaling Factor')
        axes[0].set_ylabel('Total Cost')
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(df['scaling_factor'], df['total_emissions'], marker='o', linewidth=2)
        axes[1].set_title('Impact of Transmission Capacity on Total Emissions')
        axes[1].set_xlabel('Transmission Capacity Scaling Factor')
        axes[1].set_ylabel('Total Emissions')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('transmission_sensitivity.png')
        plt.close()
        
        # Net import by country
        countries = list(df['net_import'].iloc[0].keys())
        import_data = {c: [row['net_import'][c] for _, row in df.iterrows()] for c in countries}
        
        plt.figure(figsize=(12, 6))
        for c in countries:
            plt.plot(df['scaling_factor'], import_data[c], marker='o', label=c)
        plt.title('Net Import by Country vs. Transmission Capacity')
        plt.xlabel('Transmission Capacity Scaling Factor')
        plt.ylabel('Net Import (+ import, - export)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig('net_import_transmission.png')
        plt.close()
    
    # 4. Demand Sensitivity
    if 'demand_sensitivity' in results:
        df = results['demand_sensitivity']
        
        fig, ax1 = plt.subplots(figsize=(10, 6), dpi=100)

        # Plot total cost on the left y-axis
        color1 = 'royalblue'
        ax1.plot(df['scaling_factor'], df['total_cost'] / 1e6, marker='o', linewidth=2, color=color1, label='Total Cost')
        ax1.set_xlabel('Demand Scaling Factor', fontsize=12)
        ax1.set_ylabel('Total Cost (Million €)', fontsize=12, color=color1)
        ax1.tick_params(axis='y', labelcolor=color1)
        ax1.grid(True, alpha=0.3)

        # Create a second y-axis sharing the same x-axis
        ax2 = ax1.twinx()
        color2 = 'darkorange'
        ax2.plot(df['scaling_factor'], df['total_emissions'] / 1e12, marker='o', linewidth=2, color=color2, label='Total Emissions')
        ax2.set_ylabel('Total Emissions (Gton CO₂eq)', fontsize=12, color=color2)
        ax2.tick_params(axis='y', labelcolor=color2)

        # Title and layout
        plt.title('Impact of Demand Scaling on Cost and Emissions', fontsize=14, weight='bold')
        fig.tight_layout()
        plt.savefig('demand_sensitivity.png', bbox_inches='tight')
        plt.close()
    
    # 5. Renewable Target Sensitivity
    if 'renewable_target_sensitivity' in results:
        df = results['renewable_target_sensitivity']
        df['target'] = df['target'] * 100  # Convert to percentage
        
        # Use seaborn styling
        sns.set(style="whitegrid")

        # Create figure and primary axis
        fig, ax1 = plt.subplots(figsize=(12, 6))

        # Plot total cost
        color_cost = 'tab:blue'
        ax1.plot(df['target'], df['total_cost'] / 1e6, marker='o', linestyle='-', color=color_cost, linewidth=2, label='Total Cost (mil €)')
        ax1.set_xlabel('Renewable Target (%)', fontsize=12)
        ax1.set_ylabel('Total Cost (mil €)', color=color_cost, fontsize=12)
        ax1.tick_params(axis='y', labelcolor=color_cost)
        ax1.yaxis.label.set_color(color_cost)

        # Create secondary y-axis for emissions
        ax2 = ax1.twinx()
        color_emissions = 'tab:red'
        ax2.plot(df['target'], df['total_emissions'] / 1e12, marker='s', linestyle='--', color=color_emissions, linewidth=2, label='Total Emissions (Gton CO2eq)')
        ax2.set_ylabel('Total Emissions (Gton CO2eq)', color=color_emissions, fontsize=12)
        ax2.tick_params(axis='y', labelcolor=color_emissions)
        ax2.yaxis.label.set_color(color_emissions)

        # Title and grid
        plt.title('Impact of Renewable Target on Cost and Emissions', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)

        # Legends
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left', fontsize=10)

        # Layout and save
        plt.tight_layout()
        plt.savefig('renewable_target_sensitivity.png', dpi=300)
        plt.close()
        
        # Renewable percentage by country
        countries = list(df['renewable_percentage'].iloc[0].keys())
        renewable_data = {c: [row['renewable_percentage'][c] for _, row in df.iterrows()] for c in countries}
        
        plt.figure(figsize=(12, 6))
        for c in countries:
            plt.plot(df['target'], renewable_data[c], marker='o', label=c)
        plt.plot(df['target'], df['target'], 'k--', label='Target')
        plt.title('Achieved Renewable Percentage by Country vs. Target')
        plt.xlabel('Renewable Target')
        plt.ylabel('Achieved Renewable Percentage')
        # legend in middle left
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.grid(True, alpha=0.3)
        plt.savefig('renewable_percentage_target.png')
        plt.close()

def compare_with_historical(historical_data, model_solution):
    """
    Compare model solution with historical data
    
    historical_data: Dictionary with historical generation and transmission data
    model_solution: Result from run_lp_model
    """
    # Extract relevant data
    historical_generation = historical_data['generation']
    historical_transmission = historical_data['transmission']
    
    model_generation = model_solution['generation']
    model_transmission = model_solution['transmission']
    
    # Calculate differences
    generation_diff = {}
    for key in set(historical_generation.keys()) | set(model_generation.keys()):
        if key in historical_generation and key in model_generation:
            generation_diff[key] = model_generation[key] - historical_generation[key]
    
    transmission_diff = {}
    for key in set(historical_transmission.keys()) | set(model_transmission.keys()):
        if key in historical_transmission and key in model_transmission:
            transmission_diff[key] = model_transmission[key] - historical_transmission[key]
    
    # Calculate metrics
    metrics = {
        'generation_mae': np.mean([abs(v) for v in generation_diff.values()]),
        'generation_mape': np.mean([abs(v/historical_generation[k]) for k, v in generation_diff.items() 
                                   if k in historical_generation and historical_generation[k] != 0]) * 100,
        'transmission_mae': np.mean([abs(v) for v in transmission_diff.values()]),
        'transmission_mape': np.mean([abs(v/historical_transmission[k]) for k, v in transmission_diff.items() 
                                     if k in historical_transmission and historical_transmission[k] != 0]) * 100
    }
    
    # Visualizations
    sources = list(set(s for s, _ in generation_diff.keys()))
    countries = list(set(c for _, c in generation_diff.keys()))
    
    # Generation comparison by source and country
    for c in countries:
        fig, ax = plt.subplots(figsize=(12, 6))
        source_labels = [s for s in sources if (s, c) in historical_generation or (s, c) in model_generation]
        historical_values = [historical_generation.get((s, c), 0) for s in source_labels]
        model_values = [model_generation.get((s, c), 0) for s in source_labels]
        
        x = np.arange(len(source_labels))
        width = 0.35
        
        ax.bar(x - width/2, historical_values, width, label='Historical')
        ax.bar(x + width/2, model_values, width, label='Model')
        
        ax.set_xlabel('Energy Source')
        ax.set_ylabel('Generation (MW)')
        ax.set_title(f'Generation Comparison for {c}')
        ax.set_xticks(x)
        ax.set_xticklabels(source_labels)
        ax.legend()
        
        plt.tight_layout()
        plt.savefig(f'generation_comparison_{c}.png')
        plt.close()
    
    # Transmission comparison
    fig, ax = plt.subplots(figsize=(14, 6))
    pair_labels = [f"{c1}->{c2}" for c1, c2 in set(historical_transmission.keys()) | set(model_transmission.keys())]
    historical_values = [historical_transmission.get((c1, c2), 0) for c1, c2 in 
                        [tuple(label.split('->')) for label in pair_labels]]
    model_values = [model_transmission.get((c1, c2), 0) for c1, c2 in 
                   [tuple(label.split('->')) for label in pair_labels]]
    
    x = np.arange(len(pair_labels))
    width = 0.35
    
    ax.bar(x - width/2, historical_values, width, label='Historical')
    ax.bar(x + width/2, model_values, width, label='Model')
    
    ax.set_xlabel('Country Pair')
    ax.set_ylabel('Transmission (MW)')
    ax.set_title('Transmission Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(pair_labels)
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('transmission_comparison.png')
    plt.close()
    
    return {
        'generation_diff': generation_diff,
        'transmission_diff': transmission_diff,
        'metrics': metrics
    }

# Example usage
if __name__ == "__main__":
    # Load your data and run sensitivity analysis
    # Example: 
    sensitivity_results = sensitivity_analysis(C, K, Q, L, D, E, renewable_sources, r_target)
    visualize_sensitivity_analysis(sensitivity_results)
    
    # To compare with historical data:
    # historical_data = {
    #     'generation': {...},  # Dictionary with (source, country): value pairs
    #     'transmission': {...}  # Dictionary with (from_country, to_country): value pairs
    # }
    # base_result = run_lp_model(1.0, 2.5e-5, C, K, Q, L, D, E, renewable_sources, r_target)
    # comparison_results = compare_with_historical(historical_data, base_result)
    
    print("Sensitivity analysis code ready to run.")

In [ ]:
# find actual total energy lithuania produced in 2024
lithuania_2024 = energy_type[energy_type['Country'] == 'Latvia']
lithuania_2024 = lithuania_2024[lithuania_2024['Date'] >= start_date]
lithuania_2024 = lithuania_2024[lithuania_2024['Date'] < end_date]

lithuania_2024 = lithuania_2024.groupby('Production Type')['Generation (MW)'].sum()
lithuania_2024.sum()